In [19]:
import vectorbt as vb
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [2]:
from pykrx import stock

# 볼린저밴드

In [42]:
def krxBB(df, window):
    df = df[['종가']].copy()
    std = df['종가'].rolling(window).std()
    
    df['중심선'] = df['종가'].rolling(window).mean()
    df['상단선'] = df['중심선'] + 2 * std
    df['하단선'] = df['중심선'] - 2 * std
    
    df['일간수익률'] = df['종가'].pct_change() + 1
    #종가>상단선 = 판매
    #종가<하단선 = 구매
    df.loc[df['종가'] > df['상단선'], '매매신호'] = False
    df.loc[df['종가'] < df['하단선'], '매매신호'] = True
    
    df.loc[df['매매신호'].shift(1) == True, '보유여부'] = True
    df.loc[df['매매신호'].shift(1) == False, '보유여부'] = False
    df['보유여부'].ffill(inplace=True)
    df['보유여부'].fillna(False, inplace=True)
    
    df['보유수익률'] = df.loc[ df['보유여부'] == True, '일간수익률']
    #NaN => x1
    df['보유수익률'].fillna(1, inplace=True)
    df['누적수익률'] = df['보유수익률'].cumprod()
    
    df['단순보유수익률'] = df['일간수익률'].cumprod()
    #그래프
#     df[['단순보유수익률', '누적수익률']].plot(figsize=(12, 4))
    
    delta = df.index[-1] - df.index[0]
    year = delta.days/365
    print("CAGR:", (df['누적수익률'].iloc[-1] ** (1/year)) )
    print(window, "RollAvg:", df['보유수익률'].cumprod().iloc[-1], end='\n\n')
    
    windows = np.arange(2, 34)
    
#     fast_ma, slow_ma = vb.MA.run_combs(df['종가'], window=windows, r=2, short_names=['fast', 'slow'])
    
#     entries = fast_ma.ma_crossed_above(slow_ma)
#     exits = fast_ma.ma_crossed_below(slow_ma)
    
    df1 = df
    df2 = df
    entries = df1['매매신호'].fillna(False)
    
    df2.loc[df2['매매신호'] == True, '매매신호'] = np.nan
    df2.loc[df2['매매신호'] == False, '매매신호'] = True
    exits = df2['매매신호'].fillna(False)
    
#     print("매수 횟수", (entries == True).sum())
#     print("매도 횟수", (exits == True).sum())
    
    pf_kwargs = dict(size=np.inf, fees=0.01, freq='1D')
    pf = vb.Portfolio.from_signals(df['종가'], entries, exits, **pf_kwargs)
    
    print('\n\n                          Backtest')
    print(pf.stats())
    pf.plot().show()
    
    pf.orders.records_readable
    

In [48]:
start = "20070101"
end = "20201231"
ticker = "005930"
window = 9

In [49]:
df = stock.get_market_ohlcv(start, end, ticker)
# for win in range(6, 20):
krxBB(df, window)

CAGR: 1.1517632130910977
9 RollAvg: 7.231909174742071



                          Backtest
Start                                2007-01-02 00:00:00
End                                  2020-12-30 00:00:00
Period                                3461 days 00:00:00
Start Value                                        100.0
End Value                                     331.506444
Total Return [%]                              231.506444
Benchmark Return [%]                               548.0
Max Gross Exposure [%]                             100.0
Total Fees Paid                               182.235522
Max Drawdown [%]                               34.976988
Max Drawdown Duration                  934 days 00:00:00
Total Trades                                          39
Total Closed Trades                                   39
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   53.846